In [1]:
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential # Fixed typo: tenforflow to tensorflow
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop


In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
#For training data
text = text[300000 : 800000]

In [5]:
characters = sorted(set(text))

In [6]:
char_to_text = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [7]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i : i + SEQ_LENGTH]) #Corrected the slicing syntax
  next_characters.append(text[i + SEQ_LENGTH])

In [8]:
import numpy as np
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool) # Changed np.bool to bool
y = np.zeros((len(sentences), len(characters)), dtype=bool)

In [11]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence): # Change sentences to sentence
    x[i, t, char_to_index[character]] = 1
  y[i, char_to_index[next_characters[i]]] = 1

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# Correct the argument name from 'optimizers' to 'optimizer'
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01)) # Change optimizers to optimizer
model.fit(x, y, batch_size=256, epochs=4)
# Change the filename to include the .keras extension
model.save('textgenerator.keras') # Or 'textgenerator.h5'

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 1.4784
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 1.4359
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 1.4055
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.3770


In [16]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [17]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [18]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.7))
print(generate_text(300, 0.8))

rwick:
can oxford, that did ever fence the sentent.

king richard ii:
and so much in the reconton, and the dead
and man i have in the senten the recontent.

romeo:
i have a man and strength to the seasing the crown,
and streadon my lord of sir, and stear the land
that she to speak the true for the county
to the world of the streech the cr

why, and i challenge nothing but my duke.

first untrice york:
i lord the treaming words in a mercured the lord;
and bear the word of the day the rest, or speak.

king richard ii:
he should not speak to the death he is and the king.

romeo:
but i cannot lord, that i am to sweet for i have not so in the county
the sentence in life is stre
ll cross the seas, and bid false edward are weep,
and there shall little and lies and friends;
and bear the demites of the forth stread,
the bother of your seasing greated soul the word,
in her speaking part be welcome in my soul,
and the scrift sire to my lord of one the crown,
command could faust in the county of a

In [10]:
characters = sorted(set(text))
char_to_index = dict((c, i) for i, c in enumerate(characters)) # Define char_to_index
index_to_char = dict((i, c) for i, c in enumerate(characters))

SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])  # Corrected the slicing syntax
    next_characters.append(text[i + SEQ_LENGTH])

import numpy as np

x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=bool)  # Changed np.bool to bool
y = np.zeros((len(sentences), len(characters)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):  # Iterate over the sentence instead of sentences
        x[i, t, char_to_index[character]] = 1
    y[i, char_to_index[next_characters[i]]] = 1